In [7]:
import pandas as pd
import numpy as np
from xray_stats import load_process as lp

import os
from PIL import Image

import plotly.graph_objects as go
from ipywidgets import Output, VBox
from IPython.display import display

## Comparing key soil metrics to corresponding X-Ray images
To get an intuition of what different regions of kurtosis/skewness/sobel space look like, in this notebook I provide a script that interactively lets you select individual plotted points of horizontal slices and see the x-ray image of that slice.


### Locally store smaller file size images
These select slices have been predecimated and saved as a part of this analysis package (so no need to access the entire raw x-ray dataset, see ../data/sample_images/). While these images are already provided, in this first code cell below, I provide the code that generates these decimated horizontal slices. You can skip to next cell to see the visual comparison.

In [ ]:
sample_image_write_path = "../data/sample_images_recompute"

def get_center_image(path_to_tiff_stack, filename):
    xc, yc, r = lp.estimate_circle_from_image(path_to_tiff_stack, filename)
    img = Image.open(os.path.join(path_to_tiff_stack, filename))
    img_center = np.array(img)
    img_center = img_center[(int(yc)-400):(int(yc)+400),(int(xc)-400):(int(xc)+400)]
    return img_center
def write_center_image(tiff_index,stack_index,tiff_files_sorted, path_to_tiff_stack,output_path,outer_median,container_median):
        
    # Get center image
    img_center = get_center_image(path_to_tiff_stack, tiff_files_sorted[tiff_index])
    # Normalize based on outer intensity and container intensity
    img_center = ((img_center-outer_median)/(container_median-outer_median)*1.022/2.5*255)
    img_center[img_center<0]=0
    img_center[img_center>255]=255
    img_center = img_center.astype(np.uint8)
    # Decimate the image (e.g., take every second pixel)
    decimation_factor = 4
    img_center_decimated = img_center[::decimation_factor, ::decimation_factor]
    
    # Create an output filename
    output_filename = f"t_{tiff_index}_s_{stack_index}.png"
    output_full_path = os.path.join(output_path, output_filename)
    
    # Convert and save the center image as PNG
    img_center_pil = Image.fromarray(img_center_decimated)
    img_center_pil.save(output_full_path, format="PNG")
    
stacks = list(range(54)) 
stacks = [x + 1 for x in stacks]
tiffskips = 50

datas = []
for stack in stacks:
    # For the purposes of this analysis, I ignore scan 30, 
    # since it does not have container or air pixels to 
    # normalize to density like I do for the other images.
    if stack != 30:
        # Build the array of tiff indices to sample.
        total_images, tiff_files_sorted, path_to_tiff_stack = lp.get_tiff_stack(stack)
        outer_noise_attenuation = [];
        container_attenuation = [];
        i = 0;
        # Subsample the tiffs to get average of background noise and container density
        for tiff_index in np.linspace(0, total_images - 1, num=50, dtype=int): 
            i = i+1
            try:
                image = Image.open(os.path.join(path_to_tiff_stack, tiff_files_sorted[tiff_index]))
                image = (np.array(image))
                mask_container, mask_outside, mask_core = lp.get_masks_pre(image)

                outer_noise_attenuation.append(np.nanmean(image[mask_outside]))
                container_attenuation.append(np.nanmean(image[mask_container]))
                lp.print_progress(i, 50)
            except Exception as error:
                print(f"An error occurred while calculating container and air attenuation: {error}")
        outer_median = np.median(outer_noise_attenuation)
        container_median = np.median(container_attenuation) 
        tiffs = np.arange(0, total_images, tiffskips)
        for tiff in tiffs:
            lp.print_and_flush(f"Stack: {stack}, Tiff: {tiff}")
            try:
                write_center_image(tiff,stack,tiff_files_sorted,path_to_tiff_stack,sample_image_write_path,outer_median,container_median)
            except Exception as error:
                print(f"An error occurred during image writing: {error}")

### Interactive comparison
Run the below code to see how different values of average sliding window (50px window size) skewness, kurtosis, and sobel edges affects the visuals of the horizontal xray slice. Marker color is currently based on depth of the horizontal slice (brighter is deeper).

In [8]:
# Create FigureWidget
fig = go.FigureWidget() 

# Load the data
df = pd.read_csv('../data/precomputed_soil_stats.csv')
df = df[(df["window_size"] == 50) & (df["stack_index"] != 30)]
df = df.reset_index()

# Add trace
fig.add_trace(go.Scatter3d(
  x=df["skew_mean"],
  y=df["kurt_mean"], 
  z=df["edge_mean"],
  marker_color=df["depth"],
  marker_size = df["skew_mean"]*0+7,
  mode='markers'
))
fig.data[0].update(marker=dict(opacity=1, line=dict(width=0)))
def display_image(trace, points, selector):
    
    idx = points.point_inds[0] # Use point_inds
    tiff = df.loc[idx, 'tiff_index']
    stack = df.loc[idx, 'stack_index']
    img_name = '../data/sample_images/t_'+str(int(tiff))+'_s_'+str(int(stack))+'.png'
    # Add image to figure
    fig.add_layout_image(
    dict(
    source=img_name,
    xref="paper",
    yref="paper",
    sizex=0.7,
    sizey=0.7,
    xanchor='left', 
    yanchor='bottom'
    )
    )
# Register click handler  
fig.data[0].on_hover(display_image)

# Update axis labels
fig.update_layout(scene=dict(
    xaxis_title="Skew Mean",
    yaxis_title="Kurt Mean",
    zaxis_title="Edge Mean"
))

# Display figure
VBox([fig])

    'data': [{'marker': {'color': array([ 0.   ,  0.195,  0.39 , ..., 12.48 , 12…